# Tensorflow MNIST MIFace Model Inversion demo

>⚠️ **Warning:** The attack used in this demo is computationally expensive and will take a very long to complete if run using the CPUs found in a typical personal computer.
> For this reason, it is highly recommended that you run these demos on a CUDA-compatible GPU.

This notebook contains a demonstration of using Dioptra to run experiments that investigate the model inversion attack when launched on a neural network model trained on the MNIST dataset. Please see the [example README](README.md) for instructions on how to prepare your environment for running this example.

## Setup

Below we import the necessary Python modules and ensure the proper environment variables are set so that all the code blocks will work as expected,

In [ ]:
# Import packages from the Python standard library
import importlib.util
import os
import pprint
import sys
import warnings
from pathlib import Path


def register_python_source_file(module_name: str, filepath: Path) -> None:
    """Import a source file directly.

    Args:
        module_name: The module name to associate with the imported source file.
        filepath: The path to the source file.

    Notes:
        Adapted from the following implementation in the Python documentation:
        https://docs.python.org/3/library/importlib.html#importing-a-source-file-directly
    """
    spec = importlib.util.spec_from_file_location(module_name, str(filepath))
    module = importlib.util.module_from_spec(spec)
    sys.modules[module_name] = module
    spec.loader.exec_module(module)


# Filter out warning messages
warnings.filterwarnings("ignore")

# Experiment name
EXPERIMENT_NAME = "mnist_model_inversion"

# Default address for accessing the RESTful API service
RESTAPI_ADDRESS = "http://localhost:80"

# Set DIOPTRA_RESTAPI_URI variable if not defined, used to connect to RESTful API service
if os.getenv("DIOPTRA_RESTAPI_URI") is None:
    os.environ["DIOPTRA_RESTAPI_URI"] = RESTAPI_ADDRESS

# Default address for accessing the MLFlow Tracking server
MLFLOW_TRACKING_URI = "http://localhost:35000"

# Set MLFLOW_TRACKING_URI variable, used to connect to MLFlow Tracking service
if os.getenv("MLFLOW_TRACKING_URI") is None:
    os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI

# Path to workflows archive
WORKFLOWS_TAR_GZ = Path("workflows.tar.gz")

# Register the examples/scripts directory as a Python module
register_python_source_file("scripts", Path("..", "scripts", "__init__.py"))

from scripts.client import DioptraClient
from scripts.utils import make_tar

# Import third-party Python packages
import numpy as np
from mlflow.tracking import MlflowClient

# Create random number generator
rng = np.random.default_rng(54399264723942495723666216079516778448)

## Dataset

We obtained a copy of the MNIST dataset when we ran `download_data.py` script. If you have not done so already, see [How to Obtain Common Datasets](https://pages.nist.gov/dioptra/getting-started/acquiring-datasets.html).
The training and testing images for the MNIST dataset are stored within the `/dioptra/data/Mnist` directory as PNG files that are organized into the following folder structure,

    Mnist
    ├── testing
    │   ├── 0
    │   ├── 1
    │   ├── 2
    │   ├── 3
    │   ├── 4
    │   ├── 5
    │   ├── 6
    │   ├── 7
    │   ├── 8
    │   └── 9
    └── training
        ├── 0
        ├── 1
        ├── 2
        ├── 3
        ├── 4
        ├── 5
        ├── 6
        ├── 7
        ├── 8
        └── 9

The subfolders under `training/` and `testing/` are the classification labels for the images in the dataset.
This folder structure is a standardized way to encode the label information and many libraries can make use of it, including the Tensorflow library that we are using for this particular demo.

## Submit and run jobs

The entrypoints that we will be running in this example are implemented in the Python source files under `src/` and the `src/MLproject` file.
To run these entrypoints within Dioptra's architecture, we need to package those files up into an archive and submit it to the Dioptra RESTful API to create a new job.
For convenience, we provide the `make_tar` helper function defined in `examples/scripts/utils.py`.

In [ ]:
make_tar(["src"], WORKFLOWS_TAR_GZ)

To connect with the endpoint, we will use a client class defined in the `examples/scripts/client.py` file that is able to connect with the Dioptra RESTful API using the HTTP protocol.
We connect using the client below.
The client uses the environment variable `DIOPTRA_RESTAPI_URI`, which we configured at the top of the notebook, to figure out how to connect to the Dioptra RESTful API.

In [ ]:
restapi_client = DioptraClient()

We need to register an experiment under which to collect our job runs.
The code below checks if the relevant experiment named `"mnist_model_inversion"` exists.
If it does, then it just returns info about the experiment, if it doesn't, it then registers the new experiment.

In [ ]:
response_experiment = restapi_client.get_experiment_by_name(name=EXPERIMENT_NAME)

if response_experiment is None or "Not Found" in response_experiment.get("message", []):
    response_experiment = restapi_client.register_experiment(name=EXPERIMENT_NAME)

response_experiment

Next, we need to train our model. Since it is recommended that we use GPUs, we will submit our job to the `"tensorflow_gpu"` queue. We will train one model using either the shallow net or the LeNet-5 architecture. As we are using a dedicated GPU for this example, the training time when using a LeNet-5 model will be much shorter than when using a CPU, allowing for the choice between the two model architectures.

So that we do not start this code by accident, we are embedding the code in a text block instead of keeping it in an executable code block.
**If you need to train one of the models, create a new code block and copy and paste the code into it.**

```python
response_shallow_train = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="train",
    entry_point_kwargs=" ".join([
        "-P model_architecture=shallow_net",
        "-P register_model_name=model_inversion_shallow_net",
    ]),
    queue="tensorflow_gpu",
    timeout="1h",
)

print("Training job for shallow neural network submitted")
print("")
pprint.pprint(response_shallow_train)


```python
response_le_net_train = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="train",
    entry_point_kwargs=" ".join([
        "-P model_architecture=le_net",
        "-P register_model_name=model_inversion_le_net",
    ]),
    queue="tensorflow_gpu",
    timeout="1h",
)

print("Training job for LeNet-5 neural network submitted")
print("")
pprint.pprint(response_le_net_train)


Now that we have a trained MNIST model, we can attack it with the model inversion attack. Model inversion has a number of potential inputs, including:

| parameter | data type | description |
| --- | --- | --- |
| `max_iter` | _int_ | The maximum number of iterations for gradient descent. |
| `window_length` | _int_ | The length of the window for checking whether the descent should be aborted. |
| `threshold` | _float_ | The threshold for descent stopping criterion. |
| `learning_rate` | _float_ | The learning rate for the gradient descent. |

We can attack a shallow net architecture model using the code below.

In [ ]:
response_shallow_net_miface = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="mi",
    entry_point_kwargs=" ".join(
        [
            f"-P model_name=model_inversion_shallow_net",
            "-P model_version=1",
            "-P batch_size=512",
            "-P max_iter=10000",
            "-P window_length=50",
            "-P threshold=0.99",
            "-P learning_rate=0.01",
        ]
    ),
    queue="tensorflow_gpu",
    timeout="2h",
)

pprint.pprint(response_shallow_net_miface)

Similarly, we can attack a LeNet-5 architecture model using the code below. Note that the code is the same except the entry point keyword argument being changed to use our LeNet-5 model.

In [ ]:
response_le_net_miface = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="mi",
    entry_point_kwargs=" ".join(
        [
            f"-P model_name=model_inversion_le_net",
            "-P model_version=1",
            "-P batch_size=512",
            "-P max_iter=10000",
            "-P window_length=50",
            "-P threshold=0.99",
            "-P learning_rate=0.01",
        ]
    ),
    queue="tensorflow_gpu",
    timeout="2h",
)

pprint.pprint(response_le_net_miface)